In [1]:
import duckdb
import pandas as pd

In [2]:
con = duckdb.connect()

In [3]:
con.install_extension("httpfs")
con.load_extension("httpfs")

In [4]:
con.install_extension("spatial")
con.load_extension("spatial")

In [5]:
url = "https://data.source.coop/vida/google-microsoft-osm-open-buildings/geoparquet/by_country_s2/country_iso=FRA/2594073385365405696.parquet"

In [6]:
duckdb.read_parquet(url)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬───────────┬────────────┬────────────────┬─────────────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬───────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ boundary_id │ bf_source │ confidence │ area_in_meters │        s2_id        │ country_iso │                                                                                                                                                                                                   geometry                                                                 

In [7]:
con.sql(f"FROM '{url}'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬───────────┬────────────┬────────────────┬─────────────────────┬─────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬───────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ boundary_id │ bf_source │ confidence │ area_in_meters │        s2_id        │ country_iso │                                                                                                                         geometry                                                                                                                         │ geohash  │ __index_level_0__ │                                                      bbox                                                       │
│    int64    │  var

In [ ]:
import duckdb
import pandas as pd

con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

url = "https://data.source.coop/vida/google-microsoft-osm-open-buildings/geoparquet/by_country/country_iso=AUS/AUS.parquet"
# Preview schema
print(con.sql(f"DESCRIBE SELECT * FROM read_parquet('{url}')").df())

# ✅ Create the spatial table by first reading geometry as blob,
# then converting only valid geometries to 2D.
con.sql(f"""
    CREATE OR REPLACE TABLE AW AS
    SELECT 
        * EXCLUDE geometry,
        CASE 
            WHEN geometry IS NOT NULL THEN
                ST_Force2D(
                    ST_GeomFromWKB(CAST(geometry AS BLOB))
                )
            ELSE NULL
        END AS geometry
    FROM read_parquet('{url}')
    WHERE geometry IS NOT NULL
""")

print(con.sql("SELECT COUNT(*) FROM AW").df())





         column_name                                        column_type null  \
0        boundary_id                                             BIGINT  YES   
1          bf_source                                            VARCHAR  YES   
2         confidence                                             DOUBLE  YES   
3     area_in_meters                                             DOUBLE  YES   
4              s2_id                                             BIGINT  YES   
5        country_iso                                            VARCHAR  YES   
6           geometry                                           GEOMETRY  YES   
7            geohash                                            VARCHAR  YES   
8  __index_level_0__                                             BIGINT  YES   
9               bbox  STRUCT(xmin DOUBLE, ymin DOUBLE, xmax DOUBLE, ...  YES   

    key default extra  
0  None    None  None  
1  None    None  None  
2  None    None  None  
3  None    None  None  

InvalidInputException: Invalid Input Error: Could not parse WKB input:WKB type 'UNKNOWN' is not supported! (type id: 744833629, SRID: 3404579242)
(You can use TRY_CAST instead to replace unsupported geometries with NULL)